In [1]:
import os
import random

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

from evaluator import evaluate
from data_loader import load_kdd_cup_urc, load_yahoo_A1, load_yahoo_A2, load_yahoo_A3, load_yahoo_A4, load_power_demand # Univariate Datasets
from data_loader import load_nasa, load_ecg, load_gesture, load_smd # Multivariate Datasets

from tensorflow import keras
from tensorflow.keras import layers
from tqdm.notebook import tqdm

# THESE LINES ARE FOR REPRODUCIBILITY
random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)

In [3]:
def train_gan(gan, dataset, batch_size, codings_size, dim=1, n_epochs=50):
    generator, discriminator = gan.layers
    for epoch in range(n_epochs):
        for X_batch in dataset:
            X_batch = tf.cast(X_batch, tf.float32)
            # phase 1 - training the discriminator
            noise = tf.random.normal(shape=[batch_size, codings_size, dim])
            generated_images = generator(noise)
            X_fake_and_real = tf.concat([generated_images, X_batch], axis=0)
            y1 = tf.constant([[0.]] * batch_size + [[1.]] * batch_size)
            discriminator.trainable = True
            discriminator.train_on_batch(X_fake_and_real, y1)
            # phase 2 - training the generator
            noise = tf.random.normal(shape=[batch_size, codings_size, dim])
            y2 = tf.constant([[1.]] * batch_size)
            discriminator.trainable = False
            gan.train_on_batch(noise, y2)
            
def get_gan(x_train):
    generator = keras.models.Sequential([
        layers.InputLayer(input_shape=(x_train.shape[1], x_train.shape[2])),
        keras.layers.Conv1D(128, kernel_size=3, padding='same', activation=keras.layers.LeakyReLU(0.2)),
        keras.layers.MaxPool1D(3, padding='same'),
        keras.layers.Conv1D(64, kernel_size=3, padding='same', activation=keras.layers.LeakyReLU(0.2)),
        keras.layers.MaxPool1D(3, padding='same'),
        keras.layers.BatchNormalization(),
        keras.layers.Conv1DTranspose(64, kernel_size=5, strides=2, padding="SAME", activation="selu"),
        keras.layers.BatchNormalization(),
        layers.Flatten(),
        layers.Dense(x_train.shape[1] * x_train.shape[2]),
        layers.Reshape([x_train.shape[1], x_train.shape[2]])
    ])

    discriminator = keras.models.Sequential([
        layers.InputLayer(input_shape=(x_train.shape[1], x_train.shape[2])),
        keras.layers.Conv1D(64, kernel_size=5, strides=2, padding="SAME", activation=keras.layers.LeakyReLU(0.2)),
        keras.layers.Dropout(0.4),
        keras.layers.Conv1D(128, kernel_size=5, strides=2, padding="SAME", activation=keras.layers.LeakyReLU(0.2)),
        keras.layers.Dropout(0.4),
        keras.layers.Flatten(),
        keras.layers.Dense(1, activation="sigmoid")
    ])

    gan = keras.models.Sequential([generator, discriminator])

    discriminator.compile(loss="binary_crossentropy", optimizer="adam")
    discriminator.trainable = False
    gan.compile(loss="binary_crossentropy", optimizer="adam")
    return gan

### Yahoo S5

In [4]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [5]:
for loader in [load_yahoo_A1, load_yahoo_A2, load_yahoo_A3, load_yahoo_A4]:
    datasets = loader(128, 64)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        tf.keras.backend.clear_session()

        X_train = x_trains[i]
        X_test = x_tests[i]
        
        gan = get_gan(X_train)
        dataset = tf.data.Dataset.from_tensor_slices(X_train)
        dataset = dataset.batch(128, drop_remainder=True).prefetch(1)

        train_gan(gan, dataset, 128, X_train.shape[1], X_train.shape[2])
        
        X_test_rec = gan.layers[0].predict(X_test)
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True)
    
        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])   

  0%|          | 0/67 [00:00<?, ?it/s]

  0%|          | 0/56 [00:00<?, ?it/s]

yahoo_A1 0.666666577777783 0.24999997875000166 0.6666665777777873
yahoo_A1 0.0 0.0 -0.0
yahoo_A1 0.0 0.0 -0.0
yahoo_A1 0.0 0.0 -0.0
yahoo_A1 0.0 0.0 -0.0
yahoo_A1 0.0 0.0 -0.0
yahoo_A1 0.0 0.0 -0.0
yahoo_A1 0.9999999000000052 0.49999995000000386 0.9999999166666722
yahoo_A1 0.9999999000000052 0.7499998875000145 0.9999999166666722
yahoo_A1 0.9999998500000123 0.49999990000001493 0.9999998750000131
yahoo_A1 0.9999999000000052 0.8333332265972362 0.9999998583333494
yahoo_A1 0.39999995200000177 0.12499998437500164 -0.0
yahoo_A1 0.9999998500000123 0.49999990000001493 0.9999998750000131
yahoo_A1 0.0 0.0 -0.0
yahoo_A1 0.33333329444444565 0.09999998800000123 -0.0
yahoo_A1 0.0 0.0 -0.0
yahoo_A1 0.7499999343750033 0.3194444278009265 -0.0
yahoo_A1 0.6666666000000033 0.16666666527777654 -0.0
yahoo_A1 0.6666666000000033 0.16666666527777654 -0.0
yahoo_A1 0.7499999343750033 0.1666666744444422 -0.0
yahoo_A1 0.0 0.0 -0.0
yahoo_A1 0.0 0.0 -0.0
yahoo_A1 0.9999999166666703 0.49999996666666835 0.9999999166666

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/92 [00:00<?, ?it/s]

yahoo_A3 0.33333329444444565 0.09999998800000123 -0.0
yahoo_A3 0.33333329444444565 0.09999998800000123 -0.0
yahoo_A3 0.7499999343750033 0.16666667444444216 -0.0
yahoo_A3 0.33333329444444565 0.09999998800000123 -0.0
yahoo_A3 0.7999999200000041 0.24999999166666562 -0.0
yahoo_A3 0.33333329444444565 0.09999998800000123 -0.0
yahoo_A3 0.6666666000000033 0.16666666527777654 -0.0
yahoo_A3 0.5714285142857172 0.1250000006249989 -0.0
yahoo_A3 0.0 0.0 -0.0
yahoo_A3 0.9999999000000052 0.49999995000000386 0.0
yahoo_A3 0.7999999200000041 0.24999999166666562 -0.0
yahoo_A3 0.8571427836734729 0.2500000034722198 -0.0
yahoo_A3 0.28571425306122533 0.08333332361111209 -0.0
yahoo_A3 0.6666666000000033 0.16666666527777654 -0.0
yahoo_A3 0.28571425306122533 0.08333332361111209 -0.0
yahoo_A3 0.8571427836734729 0.2500000034722198 -0.0
yahoo_A3 0.33333329444444565 0.09999998800000123 -0.0
yahoo_A3 0.4999999500000025 0.10000000133333238 -0.0
yahoo_A3 0.39999995200000177 0.12499998437500164 -0.0
yahoo_A3 0.285714253

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/82 [00:00<?, ?it/s]

yahoo_A4 0.39999995200000177 0.12499998437500164 -0.0
yahoo_A4 0.33333329444444565 0.09999998800000123 -0.0
yahoo_A4 0.39999995200000177 0.12499998437500164 -0.0
yahoo_A4 0.4999999500000025 0.10000000133333238 0.12499998437500164
yahoo_A4 0.7999999200000041 0.24999999166666562 -0.0
yahoo_A4 0.7999999360000031 0.16666667972221952 -0.0
yahoo_A4 0.4999999500000025 0.10000000133333238 -0.0
yahoo_A4 0.4999999500000025 0.10000000133333238 -0.0
yahoo_A4 0.8571427836734729 0.2500000034722198 -0.0
yahoo_A4 0.4999999375000028 0.16666664444444684 -0.0
yahoo_A4 0.9090908429752097 0.2500000140138856 -0.0
yahoo_A4 0.6666666074074105 0.1250000081249981 -0.0
yahoo_A4 0.39999995200000177 0.12499998437500164 -0.0
yahoo_A4 0.28571425306122533 0.08333332361111209 -0.0
yahoo_A4 0.28571425306122533 0.08333332361111209 -0.0
yahoo_A4 0.39999995200000177 0.20833330451389204 0.43749995468750413
yahoo_A4 0.7999999200000041 0.24999999166666562 0.49999990000001493
yahoo_A4 0.4999999375000028 0.16666664444444684 -0

In [6]:
yahoo_results = pd.DataFrame(total_scores)
yahoo_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
yahoo_A1,0.504974,0.236954,0.309524
yahoo_A2,0.314171,0.062655,0.032902
yahoo_A3,0.573313,0.187258,0.026268
yahoo_A4,0.553464,0.162568,0.030234


### NASA

In [7]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [8]:
for loader in [load_nasa]:
    datasets = loader(128, 64)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        tf.keras.backend.clear_session()

        X_train = x_trains[i]
        X_test = x_tests[i]
        
        gan = get_gan(X_train)
        dataset = tf.data.Dataset.from_tensor_slices(X_train)
        dataset = dataset.batch(128, drop_remainder=True).prefetch(1)

        train_gan(gan, dataset, 128, X_train.shape[1], X_train.shape[2])
        
        X_test_rec = gan.layers[0].predict(X_test)
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True)
        
        total_scores['dataset'].append(f'D{i+1}')
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(f'D{i+1}', np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])   

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

D1 0.261768477932986 0.13566182676713515 0.528350196170301
D2 0.28464975199693254 0.3043227591117296 0.5233555679794182


In [9]:
nasa_results = pd.DataFrame(total_scores)
nasa_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
D1,0.261768,0.135662,0.528350
D2,0.284650,0.304323,0.523356


### SMD

In [10]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [11]:
for loader in [load_smd]:
    datasets = loader(128, 64)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        tf.keras.backend.clear_session()

        X_train = x_trains[i]
        X_test = x_tests[i]
        
        gan = get_gan(X_train)
        dataset = tf.data.Dataset.from_tensor_slices(X_train)
        dataset = dataset.batch(128, drop_remainder=True).prefetch(1)

        train_gan(gan, dataset, 128, X_train.shape[1], X_train.shape[2])
        
        X_test_rec = gan.layers[0].predict(X_test)
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True)

        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])   

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

smd 0.23153690558205123 0.5718260799691193 0.5172413782842009
smd 0.2424242012855895 0.16353417466979295 0.695751620242163
smd 0.18226599319687592 0.07249635962646318 0.35093894998285413
smd 0.1739130063327107 0.20710560496380762 0.520627687742403
smd 0.7199999481600029 0.6228193985938766 0.8547625468218552
smd 0.47835047887257154 0.11003125359988719 0.048064223521040686
smd 0.29561198390945814 0.1209592096546521 0.3984547134313108
smd 0.23444973995673352 0.17261160362048433 0.36955211780170333
smd 0.21307504139204825 0.06043745297727772 0.19127268471124986
smd 0.30023092122738076 0.1590783706874283 0.2607417834763463
smd 0.12690354134994566 0.01765508589845189 0.16680557665789147
smd 0.29953914490114675 0.12156297138451184 0.434544794556826
smd 0.2635293887601403 0.1495031024277427 0.4722874289200717
smd 0.33333330277777967 0.2156452969390002 0.5278816156252074
smd 0.23999997792000163 0.21025405462423108 0.46663514650511184
smd 0.7499999343750033 0.609776013976966 0.8499999827664837
s

In [12]:
smd_results = pd.DataFrame(total_scores)
smd_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
smd,0.29219,0.204321,0.461663


### ECG

In [13]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [14]:
for loader in [load_ecg]:
    datasets = loader(64, 32)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        tf.keras.backend.clear_session()

        X_train = x_trains[i]
        X_test = x_tests[i]
        
        gan = get_gan(X_train)
        dataset = tf.data.Dataset.from_tensor_slices(X_train)
        dataset = dataset.batch(128, drop_remainder=True).prefetch(1)

        train_gan(gan, dataset, 128, X_train.shape[1], X_train.shape[2])
        
        X_test_rec = gan.layers[0].predict(X_test)
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True)

        total_scores['dataset'].append(f'D{i+1}')
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(f'D{i+1}', np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])  

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

D1 0.6666666133333363 0.5349760021940008 0.5788043401328832
D2 0.5454544892562023 0.39764073858826815 0.6871279627823951
D3 0.04545452169422684 0.0021929824138196374 0.011168384682071545
D4 0.4347825708884712 0.21898341974738095 0.7011904590770978
D5 0.49999994950000415 0.2192999988733287 0.6566923954987047
D6 0.3999999440000059 0.1895495232925521 0.6315284071222699
D7 0.0769230652544395 0.030410211209186647 0.6462513845923926
D8 0.2249999794687516 0.09430991875255643 0.45998497935228944
D9 0.421052596860575 0.16905922603957924 0.3386752100009588


In [15]:
ecg_results = pd.DataFrame(total_scores)
ecg_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
D1,0.666667,0.534976,0.578804
D2,0.545454,0.397641,0.687128
D3,0.045455,0.002193,0.011168
D4,0.434783,0.218983,0.701190
D5,0.500000,0.219300,0.656692
D6,0.400000,0.189550,0.631528
D7,0.076923,0.030410,0.646251
D8,0.225000,0.094310,0.459985
D9,0.421053,0.169059,0.338675


### Power Demand

In [16]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [17]:
for loader in [load_power_demand]:
    datasets = loader(512, 256)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        tf.keras.backend.clear_session()

        X_train = x_trains[i]
        X_test = x_tests[i]
        
        gan = get_gan(X_train)
        dataset = tf.data.Dataset.from_tensor_slices(X_train)
        dataset = dataset.batch(128, drop_remainder=True).prefetch(1)

        train_gan(gan, dataset, 128, X_train.shape[1], X_train.shape[2])
        
        X_test_rec = gan.layers[0].predict(X_test)
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True)

        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc']) 

  0%|          | 0/1 [00:00<?, ?it/s]

power_demand 0.7499999343750033 0.2916666525694449 -0.0


In [18]:
power_results = pd.DataFrame(total_scores)
power_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
power_demand,0.75,0.291667,0.0


### 2D Gesture

In [19]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [20]:
for loader in [load_gesture]:
    datasets = loader(64, 32)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        tf.keras.backend.clear_session()

        X_train = x_trains[i]
        X_test = x_tests[i]
        
        gan = get_gan(X_train)
        dataset = tf.data.Dataset.from_tensor_slices(X_train)
        dataset = dataset.batch(128, drop_remainder=True).prefetch(1)

        train_gan(gan, dataset, 128, X_train.shape[1], X_train.shape[2])
        
        X_test_rec = gan.layers[0].predict(X_test)
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True)

        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])  

  0%|          | 0/1 [00:00<?, ?it/s]

gesture 0.5238094717687121 0.5173993788885469 0.696906953603218


In [21]:
gesture_results = pd.DataFrame(total_scores)
gesture_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
gesture,0.523809,0.517399,0.696907
